# 환경설정

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 크롬창 열기

In [4]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word_list = ['노량진수산시장 횟집', '송파구 횟집', '강서구 횟집', '강남구 횟집', '노원구 횟집', '은평구 횟집']

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_5696\2096135440.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


In [5]:
key_word = ['노량진수산시장 횟집']

In [6]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [65]:
def find_search_input(key_word):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search')
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [8]:
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [9]:
# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [29]:
# 페이지 다운
def page_end():
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    body.send_keys(Keys.END)

In [10]:
def find_store_list():
    # 매장 리스트
    store_list = driver.find_elements(By.CSS_SELECTOR,'._22p-O')
    # 페이지 리스트
    next_btn = driver.find_elements(By.CSS_SELECTOR,'._2ky45 > a')
    return store_list, next_btn

In [11]:
def find_review_btn():
    sleep(2)
    time_wait(10,'a._2RG_o._tab-menu')
    menu_list = driver.find_elements(By.CSS_SELECTOR,'a._2RG_o._tab-menu')
    if len(menu_list) ==7:
        return menu_list[-1]
    else:
        return menu_list[-2]

In [34]:
def crawling_visitor_review():
    #-----리뷰 버튼 클릭-------
    find_review_btn().click()
    SCROLL_PAUSE_SEC = 1
    
    last_scroll = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        
        try :
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'._3iTUo').click()
            
        except:
            #더보기 버튼이 없다면 while문을 빠져나온다.
            break
            # 끝까지 스크롤 다운
            
        new_scroll = driver.execute_script("return document.body.scrollHeight")
        if last_scroll == new_scroll:
            break
        else:
            last_scroll= new_scroll



    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,"div.faZHB > a > span")
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return review_list


In [ ]:
def crawling_blog_review():
    SCROLL_PAUSE_SEC = 3
    sleep(SCROLL_PAUSE_SEC)
    try :
        blog_review_button = driver.find_element(By.CSS_SELECTOR, 'div._1tclc > div > a:nth-child(2)')
        blog_review_button.click()
    except:
        print('블로그 리뷰가 없습니다.')
        return
    else:
        driver.implicitly_wait(SCROLL_PAUSE_SEC)

        last_scroll = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

            try :
                #더보기 버튼 클릭
                driver.find_element(By.CSS_SELECTOR,'a._3iTUo').click()
                driver.implicitly_wait(SCROLL_PAUSE_SEC)
            except:
                break
                # 끝까지 스크롤 다운

            new_scroll = driver.execute_script("return document.body.scrollHeight")
            driver.implicitly_wait(SCROLL_PAUSE_SEC)
            if last_scroll == new_scroll:
                break
            else:
                last_scroll = new_scroll

        #가게 리스트 마다 방문자 리뷰 가져오기
        review_elements = driver.find_elements(By.CSS_SELECTOR,'div._3jvFk')
        review_list = []
        for i in review_elements:
            review_list.append(i.text)
        return review_list


# 검색 지역 횟집의 리뷰를 크롤링.

In [104]:
review_dic={}
def review_crawling(key_word_list,nums):
    WAIT=3
    for key_word in key_word_list:
        review_dic[key_word] = {}
        #검색창을 찾아서 서치
        find_search_input(key_word)
        # searchIframe 변경
        switch_frame('searchIframe')
        driver.implicitly_wait(WAIT)
        # 페이지 버튼의 수
        pages = driver.find_elements(By.CSS_SELECTOR,'a._2tk2s')
        for btn in range(nums): #페이지 버튼
            if btn != 5:
                pages[btn].click()
            else:
                driver.find_element(By.CSS_SELECTOR, 'div._2ky45 > a:last-child').click()
            
            page_down(40)
            driver.implicitly_wait(2)
            store_elements = driver.find_elements(By.CSS_SELECTOR,'div._3ZU00._1rBq3 > a:nth-child(1)')

            #한 페이지의 50개의 매장들 하나씩 조회
            for data in range(len(store_elements)):
                driver.execute_script("window.scrollTo(0, 0);")
                driver.implicitly_wait(1)
                store_elements[data].click() #li의 스토어를 클릭
                driver.implicitly_wait(WAIT) # 2초 휴식
                try:
                    #다른 iframe으로 변경
                    switch_frame('entryIframe')
                    time_wait(2, '._3XamX')

                    # -----매장명 가져오기-----
                    store_name = driver.find_element(By.CSS_SELECTOR,'._3XamX').text
                    driver.implicitly_wait(2)
                    review_dic[key_word][store_name] = {}
                    
                    # -----위치 가져오기-----
                    try : 
                        location = driver.find_element(By.CSS_SELECTOR, 'span._2yqUQ').text
                        driver.implicitly_wait(WAIT)
                        review_dic[key_word][store_name]['location'] = location
                    except:
                        pass
                    
                    # -----리뷰 크롤링---------
                    # 방문자 리뷰 크롤링
                    review_dic[key_word][store_name]['review'] = crawling_visitor_review()
                    driver.implicitly_wait(WAIT)
                    driver.execute_script("window.scrollTo(0, 0);")
                    driver.implicitly_wait(WAIT)
                    # 블로거 리뷰 크롤링
                    review_dic[key_word][store_name]['review'] = review_dic[key_word][store_name]['review']+crawling_blog_review()
                    driver.implicitly_wait(WAIT)

                except:
                    pass
                switch_frame('searchIframe')

        # json 파일로 저장
    with open('naver_review_dic.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
        
            

In [108]:
# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
#options = webdriver.ChromeOptions()
#options.add_argument('headless')
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word_list = ['서울 횟집', '인천 횟집', '부산 횟집', '대구 횟집', ' 대전 횟집', '울산 횟집', '광주 횟집']

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_5696\3255712442.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [109]:
review_crawling(key_word_list,5)

블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.
블로그 리뷰가 없습니다.


WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B378B3+2193587]
	Ordinal0 [0x00AD0681+1771137]
	Ordinal0 [0x009E4070+802928]
	Ordinal0 [0x009D9AB2+760498]
	Ordinal0 [0x009CCEAF+708271]
	Ordinal0 [0x009D1142+725314]
	Ordinal0 [0x009D55C9+742857]
	Ordinal0 [0x009E5500+808192]
	Ordinal0 [0x00A3D2C2+1168066]
	Ordinal0 [0x00A2D5C6+1103302]
	Ordinal0 [0x00A077E0+948192]
	Ordinal0 [0x00A086E6+952038]
	GetHandleVerifier [0x00DE0CB2+2738370]
	GetHandleVerifier [0x00DD21B8+2678216]
	GetHandleVerifier [0x00BC17AA+512954]
	GetHandleVerifier [0x00BC0856+509030]
	Ordinal0 [0x00AD743B+1799227]
	Ordinal0 [0x00ADBB68+1817448]
	Ordinal0 [0x00ADBC55+1817685]
	Ordinal0 [0x00AE5230+1856048]
	BaseThreadInitThunk [0x76BEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F475F4+228]
	RtlGetAppContainerNamedObjectPath [0x76F475C4+180]


In [ ]:

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)


# sample

In [ ]:
find_search_input('부산 횟집')

In [94]:
time_wait(2, 'div.input_box > input.input_search')
search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
search.clear()
key_word='서울 횟집'
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
sleep(1)

In [95]:
switch_frame('searchIframe')

In [96]:
pages = driver.find_elements(By.CSS_SELECTOR,'a._2tk2s')

In [97]:
pages

[<selenium.webdriver.remote.webelement.WebElement (session="5a679fd7690dd032f16865d4329b8699", element="4d559f8b-11c6-4f84-bb82-e26025bf2415")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5a679fd7690dd032f16865d4329b8699", element="bdd84b56-dec8-4382-bdf2-66ee85b750a4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5a679fd7690dd032f16865d4329b8699", element="373816a7-b6d5-4b12-8f6e-e1300d0cd2dd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5a679fd7690dd032f16865d4329b8699", element="a5200dbe-fe7a-4c70-b4b8-4e121576543e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5a679fd7690dd032f16865d4329b8699", element="f3f7cccd-e177-4ef8-aac0-3b9057cc70bc")>]

In [98]:
pages[3].click()

In [101]:
pages[4].click()

In [103]:
driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div._2ky45 > a:last-child').click()

In [100]:
pages[0].click()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B378B3+2193587]
	Ordinal0 [0x00AD0681+1771137]
	Ordinal0 [0x009E41A8+803240]
	Ordinal0 [0x009E6BB4+814004]
	Ordinal0 [0x009E6A72+813682]
	Ordinal0 [0x009E6D00+814336]
	Ordinal0 [0x00A13F19+999193]
	Ordinal0 [0x00A09146+954694]
	Ordinal0 [0x00A2D7AC+1103788]
	Ordinal0 [0x00A08C04+953348]
	Ordinal0 [0x00A2D9C4+1104324]
	Ordinal0 [0x00A3DAE2+1170146]
	Ordinal0 [0x00A2D5C6+1103302]
	Ordinal0 [0x00A077E0+948192]
	Ordinal0 [0x00A086E6+952038]
	GetHandleVerifier [0x00DE0CB2+2738370]
	GetHandleVerifier [0x00DD21B8+2678216]
	GetHandleVerifier [0x00BC17AA+512954]
	GetHandleVerifier [0x00BC0856+509030]
	Ordinal0 [0x00AD743B+1799227]
	Ordinal0 [0x00ADBB68+1817448]
	Ordinal0 [0x00ADBC55+1817685]
	Ordinal0 [0x00AE5230+1856048]
	BaseThreadInitThunk [0x76BEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F475F4+228]